# Info

You can run the entire script by press **CTRL+F9**.

For more helpful scripts, check out the quick commands.
*   [Robohood | Quick commands](https://colab.research.google.com/drive1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5?usp=sharing)
*  [Cancel all pending orders](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=ElxYAXz3LWR5&line=1&uniqifier=1)
*  [Close all positions](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=LD2LSDnqOlFe&line=1&uniqifier=1)

#Application code
Only needs to be run once per session.

In [19]:
%pip install robin_stocks

Note: you may need to restart the kernel to use updated packages.


In [20]:
from robin_stocks import robinhood as r
import itertools
from datetime import datetime
import getpass
import sys

## Funcitons

In [21]:
def get_spread_cost(front_leg_option, back_leg_option):
  spread_cost = sys.maxsize
  back_ask_price = back_leg_option.get("high_fill_rate_buy_price")
  front_bid_price = front_leg_option.get("high_fill_rate_sell_price")

  if(back_ask_price and front_bid_price):
    spread_cost = round(float(back_ask_price) - float(front_bid_price), 2)

  return spread_cost

In [22]:
def find_calendar_spreads(options):
    # Create a create new calendar spread for every combination of expiration dates where the short front leg expiration_date is less than the long back leg of the spread
  sorted_options = sorted(options, key=lambda x: (float(x['strike_price']), datetime.strptime(x["expiration_date"],'%Y-%m-%d')))

  options_grouped_by_strike = itertools.groupby(sorted_options, lambda x : x['strike_price'])

  today = datetime.today()

  #  using the front_leg_option and back_leg_option as the front and back legs of the trade, create a debit calendar spread
  #  with a cost of zero
  calendar_spreads = []
  for strike, options in options_grouped_by_strike:
    for (front_leg_option, back_leg_option) in itertools.combinations(options,2):
      front_expiration_date = datetime.strptime(front_leg_option["expiration_date"],'%Y-%m-%d')
      back_expiration_date = datetime.strptime(back_leg_option["expiration_date"],'%Y-%m-%d')
      optionTypeIsSame = front_leg_option["type"] == back_leg_option["type"]

      if (optionTypeIsSame and (front_expiration_date > today) and (front_expiration_date < back_expiration_date)):
        calendar_spreads.append((front_leg_option, back_leg_option, get_spread_cost(front_leg_option, back_leg_option)))

  return calendar_spreads

In [23]:
def order_calendar_spread(spread, price=0.00, quantity=1, timeInForce='gfd'):
  front_leg_option, back_leg_option, spread_cost = spread
  symbol = front_leg_option["symbol"]
  params = [
            {
            'expirationDate': front_leg_option['expiration_date'],
            'strike': front_leg_option['strike_price'],
            'optionType': front_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'sell',
          },
          {
            'expirationDate': back_leg_option['expiration_date'],
            'strike': back_leg_option['strike_price'],
            'optionType': back_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'buy',
          },
  ]
  
  return r.orders.order_option_spread(direction='debit', price=price, symbol=symbol, quantity=quantity, spread=params, timeInForce=timeInForce)


In [24]:
def filter_calendar_spreads(spreads, spread_max_cost=.1):

  filtered_spreads = list(filter(lambda spread: spread[2] <= spread_max_cost, spreads))
  filtered_spreads = sorted(filtered_spreads, key=lambda spread: (spread[2]))

  return filtered_spreads

In [25]:
def get_current_price_of_symbol(symbol="SPY"):
  current_price = round(float(r.stocks.get_latest_price(symbol)[0]))
  print("{} is currently trading at: ${}".format(symbol, current_price))
  return current_price 

In [26]:
def scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset):
  if(optionType == 'both'):
    optionType = ""

  current_price = get_current_price_of_symbol(symbol)
  min_strike = current_price - min_strike_offset;
  max_strike = current_price + max_strike_offset;

  # Get the list of options available and sort by strike_price and expiration_date
  options = r.options.find_tradable_options(symbol, optionType=optionType, info=None)
  options = list(filter(lambda x: (float(x["strike_price"]) > min_strike) and (float(x["strike_price"]) < max_strike), options))
  return options

In [27]:
def print_spread(spread):
  (front_leg_option, back_leg_option, spread_cost) = spread
  strike = float(front_leg_option.get("strike_price"))
  print("strike {} | short {} @ {} | long {} @ {} | spread: ${}".format(
      strike,
      front_leg_option.get("type"),
      front_leg_option.get("expiration_date"),
      back_leg_option.get("type"),
      back_leg_option.get("expiration_date"),
      spread_cost))

In [28]:
def print_spreads(spreads):
  for spread in spreads:
    print_spread(spread)

In [55]:
def place_calendar_spreads(spreads, price_type, quanity_type, timeInForce):
  price = 0.00
  quantity=1
  results = []
  if(price_type == "set_for_all"):
    price = float(input("Price for all orders: ") or 0.00)

  if(quanity_type == "set_for_all"):
    quantity = int(input("Quantity for all orders: ") or 0)

  for spread in spreads:
    (front_leg_option, back_leg_option, spread_cost) = spread
    print_spread(spread)

    if(price_type == "set_each"):
      price = float(input("price (limit): ") or 0.00)

    if(price_type == "spread_cost"):
      price = spread_cost

    if(quanity_type == "set_each"):
      quantity = int(input("order size (quantity): ") or 1)

    print("buy {} @ ${}".format(quantity, price))
    result = order_calendar_spread(spread=spread, price=price, quantity=quantity, timeInForce=timeInForce)
    results.append(result)
  
  return results


In [30]:
def update_option_market_data(options):
  for item in options:
    marketData = r.options.get_option_market_data_by_id(item['id'])
    if marketData:
        item.update(marketData[0])

# Login

In [31]:
#@title #Authenticate { vertical-output: true, display-mode: "form" }
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
login = r.login(username, password)
print(login.get("detail"))

logged in using authentication in robinhood.pickle


# Scan for options

> Inputs needed: offset and max offset. By default, the scanner will analize $10 worth of different strikes. 5 above and 5 below.

* **symbol**: example "spy"
* **min strike offset**: how far below the current strike you want to look. The default is 5 dollars.
* **max strike offset**: how far above the current strike you want to look. The 
default is 5 dollars.

In [47]:
#@title Enter option criteria { run: "auto", vertical-output: true, display-mode: "form" }
# Create a script that places a debit calendar spread for each tradeable option of a given symbol. The cost of the spread needs to be zero.
symbol = "SPY" #@param {type:"string"}
optionType = "both" #@param ["call", "put", "both"]
min_strike_offset = 7 #@param {type:"number"}
max_strike_offset =  7 #@param {type:"number"}

options = scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset)
print("Base on user criteria, you found {} to analize!".format(len(options)))

SPY is currently trading at: $395
Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
Loading page 31 ...
Loading page 32 ...
Loading page 33 ...
Loading page 34 ...
Loading page 35 ...
Loading page 36 ...
Loading page 37 ...
Loading page 38 ...
Loading page 39 ...
Loading page 40 ...
Loading page 41 ...
Loading page 42 ...
Loading page 43 ...
Loading page 44 ...
Loading page 45 ...
Loading page 46 ...
Loading page 47 ...
Loading page 48 ...
Loading pa

## Get option market data


> This gets the latest market information for each option found. Data such as ask_price and bid_price are changing all the time. It's a good idea to refresh this often.

In [48]:
update_option_market_data(options)

# Filter calendar spreads

Enter max price you want filter for each spread. This is calculated by subtracting the front leg bid price from the ask price of the back leg. ask_price - bid_price = spread_cost. The default is $0.30.

In [50]:
#@title Filter based on spread cost { run: "auto", vertical-output: true, display-mode: "form" }
max_spread_cost =  0.25 #@param {type:"number"}
calendar_spreads = find_calendar_spreads(options)
filtered_spreads = filter_calendar_spreads(calendar_spreads, max_spread_cost)
print("{} spreads found.".format(len(filtered_spreads)))
print_spreads(filtered_spreads)

65 spreads found.
strike 400.0 | short put @ 2025-01-17 | long put @ 2025-03-21 | spread: $-0.25
strike 401.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $-0.21
strike 401.0 | short put @ 2023-01-20 | long put @ 2023-01-23 | spread: $-0.12
strike 400.0 | short put @ 2023-01-24 | long put @ 2023-01-25 | spread: $-0.07
strike 398.0 | short put @ 2023-01-25 | long put @ 2023-01-26 | spread: $-0.03
strike 398.0 | short put @ 2023-01-19 | long put @ 2023-01-20 | spread: $-0.02
strike 400.0 | short call @ 2023-12-15 | long call @ 2023-12-29 | spread: $-0.01
strike 391.0 | short put @ 2023-01-25 | long put @ 2023-01-26 | spread: $0.03
strike 392.0 | short put @ 2023-01-26 | long put @ 2023-01-27 | spread: $0.04
strike 394.0 | short call @ 2023-01-26 | long call @ 2023-01-27 | spread: $0.04
strike 390.0 | short call @ 2023-01-19 | long call @ 2023-01-20 | spread: $0.07
strike 393.0 | short put @ 2023-01-25 | long put @ 2023-01-26 | spread: $0.08
strike 399.0 | short put @ 2023-0

# Create calendar spread orders

Select the number of order you want to place. Note: the max number of orders you can create in robinhood is 11.

In [51]:
#@title How many orders do you want to place? { run: "auto", vertical-output: true, display-mode: "form" }
max_order_size = 11 #@param {type:"integer"}
top_filtered_spreads = filtered_spreads[:max_order_size] # robinhood only allows 12 orders
print_spreads(top_filtered_spreads)

strike 400.0 | short put @ 2025-01-17 | long put @ 2025-03-21 | spread: $-0.25
strike 401.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $-0.21
strike 401.0 | short put @ 2023-01-20 | long put @ 2023-01-23 | spread: $-0.12
strike 400.0 | short put @ 2023-01-24 | long put @ 2023-01-25 | spread: $-0.07
strike 398.0 | short put @ 2023-01-25 | long put @ 2023-01-26 | spread: $-0.03
strike 398.0 | short put @ 2023-01-19 | long put @ 2023-01-20 | spread: $-0.02
strike 400.0 | short call @ 2023-12-15 | long call @ 2023-12-29 | spread: $-0.01
strike 391.0 | short put @ 2023-01-25 | long put @ 2023-01-26 | spread: $0.03
strike 392.0 | short put @ 2023-01-26 | long put @ 2023-01-27 | spread: $0.04
strike 394.0 | short call @ 2023-01-26 | long call @ 2023-01-27 | spread: $0.04
strike 390.0 | short call @ 2023-01-19 | long call @ 2023-01-20 | spread: $0.07


In [56]:
#@title Place calendar orders based on the following parameters: { vertical-output: true, display-mode: "form" }
price_type = "set_for_all"  # @param ["set_for_all", "set_for_all", "set_each"]
quanity_type = "set_each"  # @param ["set_each", "set_for_all"]
timeInForce='gtc'  #@param ["gfd", "gtc"] 
place_calendar_spreads(top_filtered_spreads, price_type, quanity_type, timeInForce)

strike 400.0 | short put @ 2025-01-17 | long put @ 2025-03-21 | spread: $-0.25
buy 1 @ $0.0
strike 401.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $-0.21
buy 1 @ $0.0
strike 401.0 | short put @ 2023-01-20 | long put @ 2023-01-23 | spread: $-0.12
buy 1 @ $0.0
strike 400.0 | short put @ 2023-01-24 | long put @ 2023-01-25 | spread: $-0.07
buy 1 @ $0.0
strike 398.0 | short put @ 2023-01-25 | long put @ 2023-01-26 | spread: $-0.03
buy 1 @ $0.0
strike 398.0 | short put @ 2023-01-19 | long put @ 2023-01-20 | spread: $-0.02
buy 1 @ $0.0
strike 400.0 | short call @ 2023-12-15 | long call @ 2023-12-29 | spread: $-0.01
buy 1 @ $0.0
strike 391.0 | short put @ 2023-01-25 | long put @ 2023-01-26 | spread: $0.03
buy 1 @ $0.0
strike 392.0 | short put @ 2023-01-26 | long put @ 2023-01-27 | spread: $0.04
buy 1 @ $0.0
strike 394.0 | short call @ 2023-01-26 | long call @ 2023-01-27 | spread: $0.04
buy 1 @ $0.0
Error in request_post: Expecting value: line 1 column 1 (char 0)
strike 390.0 |

[{'account_number': '5UI31739',
  'cancel_url': 'https://api.robinhood.com/options/orders/63c026da-ea3c-4b1c-8294-0630f542acc6/cancel/',
  'canceled_quantity': '0.00000',
  'created_at': '2023-01-12T15:27:22.564897Z',
  'direction': 'debit',
  'id': '63c026da-ea3c-4b1c-8294-0630f542acc6',
  'legs': [{'executions': [],
    'id': '63c026da-12c7-46f2-a159-2ae4001c55f3',
    'option': 'https://api.robinhood.com/options/instruments/2b6e4188-cf1e-494d-b1e6-3894fee057b3/',
    'position_effect': 'open',
    'ratio_quantity': 1,
    'side': 'buy',
    'expiration_date': '2025-03-21',
    'strike_price': '400.0000',
    'option_type': 'put',
    'long_strategy_code': '2b6e4188-cf1e-494d-b1e6-3894fee057b3_L1',
    'short_strategy_code': '2b6e4188-cf1e-494d-b1e6-3894fee057b3_S1'},
   {'executions': [],
    'id': '63c026da-925a-483a-abdd-c1cf26566ab1',
    'option': 'https://api.robinhood.com/options/instruments/4356f723-12bf-4b0e-a5e8-faa9a14f1b31/',
    'position_effect': 'open',
    'ratio_quan